In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("streaming_example").getOrCreate()

In [3]:
#reading data from the TCP sockets
streaming_df = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

streaming_df.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
#applying transformation on the data (to get count of words)
from pyspark.sql.functions import split, col

words_df = streaming_df.select(
    split(col("value"), " ").alias("words")
)

exploded_df = words_df.selectExpr("explode(words) as word")

word_counts = exploded_df.groupBy("word").count()


In [5]:
word_counts.printSchema()

root
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [ ]:
#writing data to the specified sink
query = word_counts.writeStream.outputMode("complete").format("console")\
.option("checkpointLocation", "C:/Users/acer/projects/checkpoint")\
.trigger(processingTime="1 second")\
.start()
query.awaitTermination()

In [ ]:
#stoping query using function
query.stop()

displaying on notebook

In [ ]:
#import time
# query = word_counts.writeStream \
#     .outputMode("complete") \
#     .format("memory") \
#     .option("checkpointLocation", "C:/Users/acer/projects/checkpoint")\
#     .queryName("word_counts") \
#     .start()

# for _ in range(4): 
#     spark.sql("SELECT * FROM word_counts").show()
#     time.sleep(1) 
# query.stop()